In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import torch
from cases.paren_checker import HighLevelParensBalanceChecker, test_HL_parens_balancer_components, BalancedParensDataset
from cases.left_greater import HighLevelLeftGreater, test_HL_left_greater_components, LeftGreaterDataset
from cases.duplicate_remover import HighLevelDuplicateRemover, test_HL_duplicate_remover_components, DuplicateRemoverDataset
from poly_hl_model import PolyHLModel, PolyModelDataset

from iit.model_pairs.strict_iit_model_pair import StrictIITModelPair
from iit.utils.index import Ix
import torch

# Paren Checker

In [12]:
test_HL_parens_balancer_components()

All Balance tests passed!


True

In [13]:
hl_model = HighLevelParensBalanceChecker()
corr = hl_model.get_correspondence()
dataset = BalancedParensDataset(N_samples=5_000, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()

KeyboardInterrupt: 

In [ ]:
print(dataset.get_dataset().shape)
print(dataset.get_dataset()[:10]['tokens'])
print(dataset.get_dataset()[:10]['labels'])
for i in range(10):
    tokens, labels, hl_outputs = dataset.get_dataset()[i]['tokens'], dataset.get_dataset()[i]['labels'], hl_model((torch.tensor(dataset.get_dataset()[i]['tokens'])[None,:], None, None))
    nonzero = (torch.tensor(labels) - hl_outputs[0].cpu()).nonzero()
    if nonzero.numel() > 0:
        print(tokens, torch.unique(nonzero[:,0]))
        bad_indices = torch.unique(nonzero[:,0]).tolist()
        for idx in bad_indices:
            print(labels[idx], hl_outputs[0,idx])

(5000, 4)
[[3, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0], [3, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0], [3, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1], [3, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1], [3, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1], [3, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0], [3, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [3, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0], [3, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0]]
[[[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0.0, 0.0, 0.0], [1.0, 0

In [ ]:


n_epochs = 100
training_args = {
    "batch_size": 256,
    "lr": 0.001,
    "num_workers": 0,
    "use_single_loss": False,
    "behavior_weight": 0., #basically doubles the strict weight's job.
    "iit_weight": 1.,
    "strict_weight": 1.0,
    "clip_grad_norm": 1.0,
    "iit_weight_schedule" : lambda s, i: s,
    "strict_weight_schedule" : lambda s, i: s,
    "behavior_weight_schedule" : lambda s, i: s, #0.955*s if 0.955**i > 0.01 else s, #have behavior weight decay over time
    "early_stop" : True,
    "lr_scheduler": torch.optim.lr_scheduler.LinearLR,
    "scheduler_kwargs": dict(start_factor=1, end_factor=0, total_iters=n_epochs),
    "scheduler_val_metric": ["val/accuracy", "val/IIA"], #for ReduceLRonPlateau
    "scheduler_mode": "max", #for ReduceLRonPlateau
}

class LastTokenStrictIITModelPair(StrictIITModelPair):
    def get_label_idxs(self):
        return Ix[[None,-1]]

    # def loss_fn(self) -> Callable[[torch.Tensor, torch.Tensor], torch.Tensor]:
    #     return torch.nn.CrossEntropyLoss()

ll_model = hl_model.get_ll_model()
model_pair = StrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)

In [ ]:
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    optimizer_kwargs=dict(weight_decay=1e-4),
)

training_args={'batch_size': 256, 'lr': 0.001, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0, 'total_iters': 100}, 'clip_grad_norm': 1.0, 'seed': 0, 'detach_while_caching': True, 'iit_weight_schedule': <function <lambda> at 0x1041b6ca0>, 'strict_weight_schedule': <function <lambda> at 0x1041b68e0>, 'behavior_weight_schedule': <function <lambda> at 0x1677a6c00>, 'atol': 0.05, 'use_single_loss': False, 'iit_weight': 1.0, 'behavior_weight': 0.0, 'strict_weight': 1.0}


Epoch 1: lr: 9.90e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 4.07e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 3.60e-01, val/iit_loss: 1.82e-01, val/IIA: 95.50, val/accuracy: 95.16, val/strict_accuracy: 95.16
Epoch 2: lr: 9.80e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.68e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 1.29e-01, val/iit_loss: 1.33e-01, val/IIA: 95.79, val/accuracy: 96.67, val/strict_accuracy: 96.67
Epoch 3: lr: 9.70e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.45e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 7.62e-02, val/iit_loss: 1.09e-01, val/IIA: 96.75, val/accuracy: 98.90, val/strict_accuracy: 98.90
Epoch 4: lr: 9.60e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.19e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 5.31e-02, val/

# Left > Right

In [ ]:
from cases.left_greater import HighLevelLeftGreater, test_HL_left_greater_components, LeftGreaterDataset

test_HL_left_greater_components()

here
All left greater tests passed!


True

In [ ]:
hl_model = HighLevelLeftGreater()
corr = hl_model.get_correspondence()
dataset = LeftGreaterDataset(N_samples=1_000, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()
print(hl_model.hook_dict)
print(list(corr.keys()))

here
making IIT dataset
{'input_hook': HookPoint(), 'paren_counts_hook': HookPoint(), 'mlp0_hook': HookPoint()}
[input_hook, paren_counts_hook, mlp0_hook]


In [ ]:
ll_model = hl_model.get_ll_model()
model_pair = StrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    optimizer_kwargs=dict(weight_decay=1e-4),
)

training_args={'batch_size': 256, 'lr': 0.001, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0, 'total_iters': 100}, 'clip_grad_norm': 1.0, 'seed': 0, 'detach_while_caching': True, 'iit_weight_schedule': <function <lambda> at 0x1041b6ca0>, 'strict_weight_schedule': <function <lambda> at 0x1041b68e0>, 'behavior_weight_schedule': <function <lambda> at 0x1677a6c00>, 'atol': 0.05, 'use_single_loss': False, 'iit_weight': 1.0, 'behavior_weight': 0.0, 'strict_weight': 1.0}


Epoch 1: lr: 9.90e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.25e+00, train/behavior_loss: 0.00e+00, train/strict_loss: 1.23e+00, val/iit_loss: 9.13e-01, val/IIA: 57.05, val/accuracy: 52.19, val/strict_accuracy: 52.19
Epoch 2: lr: 9.80e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 8.54e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 8.03e-01, val/iit_loss: 7.66e-01, val/IIA: 63.89, val/accuracy: 70.82, val/strict_accuracy: 70.82
Epoch 3: lr: 9.70e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 7.00e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 6.53e-01, val/iit_loss: 6.86e-01, val/IIA: 67.87, val/accuracy: 80.80, val/strict_accuracy: 80.80
Epoch 4: lr: 9.60e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 5.85e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 5.47e-01, val/

# Duplicate remover
case 19 in circuits-bench

In [ ]:
from cases.duplicate_remover import HighLevelDuplicateRemover, test_HL_duplicate_remover_components, DuplicateRemoverDataset
test_HL_duplicate_remover_components()

[[4, 0, 0, 1, 2, 0, 1, 3, 3], [4, 0, 1, 2, 2, 2, 2, 2, 2], [4, 0, 1, 2, 3, 3, 3, 3, 3]]
[[False, False, True, False, False, False, False, False, True], [False, False, False, False, True, True, True, True, True], [False, False, False, False, False, True, True, True, True]]
here
All DuplicateRemover tests passed!


True

In [ ]:
hl_model = HighLevelDuplicateRemover()
corr = hl_model.get_correspondence()
dataset = DuplicateRemoverDataset(N_samples=1_000, n_ctx=hl_model.get_ll_model_cfg().n_ctx, seed=42)
train_set, test_set = dataset.get_IIT_train_test_set()
print(hl_model.hook_dict)
print(list(corr.keys()))

here
here
(1000, 15, 5)
making IIT dataset
{'input_hook': HookPoint(), 'prev_token_hook': HookPoint(), 'prev_equal_hook': HookPoint(), 'output_hook': HookPoint()}
[input_hook, prev_token_hook, prev_equal_hook, output_hook]


In [ ]:
ll_model = hl_model.get_ll_model()
model_pair = StrictIITModelPair(hl_model=hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    optimizer_kwargs=dict(weight_decay=1e-4),
)

training_args={'batch_size': 256, 'lr': 0.001, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>, 'scheduler_val_metric': ['val/accuracy', 'val/IIA'], 'scheduler_mode': 'max', 'scheduler_kwargs': {'start_factor': 1, 'end_factor': 0, 'total_iters': 100}, 'clip_grad_norm': 1.0, 'seed': 0, 'detach_while_caching': True, 'iit_weight_schedule': <function <lambda> at 0x1041b6ca0>, 'strict_weight_schedule': <function <lambda> at 0x1041b68e0>, 'behavior_weight_schedule': <function <lambda> at 0x1677a6c00>, 'atol': 0.05, 'use_single_loss': False, 'iit_weight': 1.0, 'behavior_weight': 0.0, 'strict_weight': 1.0}


Epoch 1: lr: 9.90e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.63e+00, train/behavior_loss: 0.00e+00, train/strict_loss: 1.55e+00, val/iit_loss: 1.30e+00, val/IIA: 37.89, val/accuracy: 38.26, val/strict_accuracy: 38.26
Epoch 2: lr: 9.80e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.37e+00, train/behavior_loss: 0.00e+00, train/strict_loss: 1.18e+00, val/iit_loss: 1.32e+00, val/IIA: 42.28, val/accuracy: 60.10, val/strict_accuracy: 60.10
Epoch 3: lr: 9.70e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 1.12e+00, train/behavior_loss: 0.00e+00, train/strict_loss: 9.40e-01, val/iit_loss: 1.22e+00, val/IIA: 45.33, val/accuracy: 65.09, val/strict_accuracy: 65.09
Epoch 4: lr: 9.60e-04, iit_weight: 1.00e+00, behavior_weight: 0.00e+00, strict_weight: 1.00e+00, train/iit_loss: 9.58e-01, train/behavior_loss: 0.00e+00, train/strict_loss: 7.70e-01, val/

# Unique token extraction
case 21 in circuits-bench

In [ ]:
import torch
print(torch.randint(0, 3, (10, 2)))

a = [1, 2, 3]
a.remove(2)
print(a)

tensor([[2, 2],
        [2, 1],
        [0, 0],
        [1, 2],
        [0, 2],
        [2, 0],
        [2, 2],
        [0, 2],
        [1, 0],
        [2, 0]])
[1, 3]


# Increasing / Decreasing detector
case 13 in circuits-bench

# Poly Model

### DuplicateRemover + LeftGreater

In [19]:
cases = [HighLevelDuplicateRemover, HighLevelLeftGreater]
poly_hl_model = PolyHLModel(hl_classes=cases, size_expansion=2)
corr = poly_hl_model.get_correspondence()
for k, v in corr.items():
    print(k, v)

print()
for k, v in poly_hl_model.corr_mapping.items():
    print(k, v)

input_hook {LLNode(name='blocks.0.hook_resid_pre', index=[:], subspace=None)}
mlp_hooks.0 {LLNode(name='blocks.0.mlp.hook_post', index=[:], subspace=None)}
attn_hooks.0.0 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 0, :], subspace=None)}
attn_hooks.0.1 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 1, :], subspace=None)}
task_hook {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 2, :], subspace=None)}
mlp_hooks.1 {LLNode(name='blocks.1.mlp.hook_post', index=[:], subspace=None)}

blocks.0.mlp.hook_post [[prev_equal_hook], [mlp0_hook]]
blocks.0.attn.hook_z.0 [[prev_token_hook], None]
blocks.0.attn.hook_z.1 [None, [paren_counts_hook]]
blocks.0.attn.hook_z.2 [None, None]
blocks.0.attn.hook_z.3 [None, None]
blocks.1.mlp.hook_post [[output_hook], None]
blocks.1.attn.hook_z.0 [None, None]
blocks.1.attn.hook_z.1 [None, None]
blocks.1.attn.hook_z.2 [None, None]
blocks.1.attn.hook_z.3 [None, None]


In [20]:
dataset1 = DuplicateRemoverDataset(N_samples=1_000, n_ctx=15, seed=42)
dataset2 = LeftGreaterDataset(N_samples=1_000, n_ctx=15, seed=42)
dsets = [dataset1,dataset2]
poly_dataset = PolyModelDataset(dsets, n_ctx=poly_hl_model.cfg.n_ctx)

n_samples = 100
input = poly_dataset.get_dataset()[:n_samples][0]
output = poly_hl_model((input, None, None)).cpu()
expected = poly_dataset.get_dataset()[:n_samples][1]
print(input)
print(torch.allclose(output, expected))

(1000, 15, 5)


/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/cases/duplicate_remover.py:161: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _, cache = hl_model.run_with_cache((t.tensor(sample).unsqueeze(0), None, None))


(1000, 16, 5)
tensor([[0, 4, 1,  ..., 0, 2, 1],
        [0, 4, 0,  ..., 2, 2, 0],
        [0, 4, 0,  ..., 0, 2, 2],
        ...,
        [1, 3, 1,  ..., 1, 0, 1],
        [0, 4, 0,  ..., 2, 0, 1],
        [1, 3, 0,  ..., 0, 1, 1]])
True


/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/poly_hl_model.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens = torch.cat([torch.tensor(dataset.tokens) for dataset in datasets], dim=0)
/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/poly_hl_model.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.cat([torch.tensor(dataset.labels) for dataset in datasets], dim=0)
/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/cases/utils.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sou

In [23]:
ll_model = poly_hl_model.get_ll_model()
corr = poly_hl_model.get_correspondence()
for k, v in corr.items():
    print(k, v)

n_epochs = 100
training_args = {
    "batch_size": 256,
    "lr": 3e-4,
    "num_workers": 0,
    "use_single_loss": False,
    "behavior_weight": 1., #basically doubles the strict weight's job.
    "iit_weight": 1.,
    "strict_weight": 0.4,
    "clip_grad_norm": 1.0,
    "iit_weight_schedule" : lambda s, i: s,
    "strict_weight_schedule" : lambda s, i: s,
    "behavior_weight_schedule" : lambda s, i: s, #0.955*s if 0.955**i > 0.01 else s, #have behavior weight decay over time
    "early_stop" : True,
    "lr_scheduler": torch.optim.lr_scheduler.LinearLR,
    "scheduler_kwargs": dict(start_factor=1, end_factor=0, total_iters=n_epochs),
    "scheduler_val_metric": ["val/accuracy", "val/IIA"], #for ReduceLRonPlateau
    "scheduler_mode": "max", #for ReduceLRonPlateau
}
train_set, test_set = poly_dataset.get_IIT_train_test_set()
model_pair = StrictIITModelPair(hl_model=poly_hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
print(model_pair.nodes_not_in_circuit)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    optimizer_kwargs=dict(weight_decay=1e-4),
)

input_hook {LLNode(name='blocks.0.hook_resid_pre', index=[:], subspace=None)}
mlp_hooks.0 {LLNode(name='blocks.0.mlp.hook_post', index=[:], subspace=None)}
attn_hooks.0.0 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 0, :], subspace=None)}
attn_hooks.0.1 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 1, :], subspace=None)}
task_hook {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 2, :], subspace=None)}
mlp_hooks.1 {LLNode(name='blocks.1.mlp.hook_post', index=[:], subspace=None)}
[LLNode(name='blocks.0.attn.hook_z', index=[:, :, 3, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 0, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 1, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 2, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 3, :], subspace=None)]
training_args={'batch_size': 256, 'lr': 0.0003, 'num_workers': 0, 'early_stop': True, 'lr_scheduler': <class 'torch.optim.lr_scheduler.LinearLR'>,

Epoch 1: lr: 2.97e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 4.00e-01, train/iit_loss: 1.33e+00, train/behavior_loss: 1.20e+00, train/strict_loss: 2.04e-01, val/iit_loss: 1.18e+00, val/IIA: 48.18, val/accuracy: 59.36, val/strict_accuracy: 57.45
Epoch 2: lr: 2.94e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 4.00e-01, train/iit_loss: 1.02e+00, train/behavior_loss: 6.92e-01, train/strict_loss: 1.24e-01, val/iit_loss: 1.18e+00, val/IIA: 51.01, val/accuracy: 77.90, val/strict_accuracy: 75.74
Epoch 3: lr: 2.91e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 4.00e-01, train/iit_loss: 5.70e-01, train/behavior_loss: 4.50e-01, train/strict_loss: 8.63e-02, val/iit_loss: 4.83e-01, val/IIA: 81.24, val/accuracy: 87.92, val/strict_accuracy: 84.16
Epoch 4: lr: 2.88e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 4.00e-01, train/iit_loss: 1.04e+00, train/behavior_loss: 3.39e-01, train/strict_loss: 6.02e-02, val/

## Duplicate Remover + LeftGreater + ParensBalancer

In [27]:
cases = [HighLevelDuplicateRemover, HighLevelLeftGreater, HighLevelParensBalanceChecker]
poly_hl_model = PolyHLModel(hl_classes=cases, size_expansion=2)
corr = poly_hl_model.get_correspondence()

dataset1 = DuplicateRemoverDataset(N_samples=1_000, n_ctx=15, seed=42)
dataset2 = LeftGreaterDataset(N_samples=1_000, n_ctx=15, seed=42)
dataset3 = BalancedParensDataset(N_samples=1_000, n_ctx=15, seed=42)
dsets = [dataset1,dataset2, dataset3]
poly_dataset = PolyModelDataset(dsets, n_ctx=poly_hl_model.cfg.n_ctx)


(1000, 15, 5)


/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/cases/duplicate_remover.py:161: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  _, cache = hl_model.run_with_cache((t.tensor(sample).unsqueeze(0), None, None))


(1000, 16, 5)


/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/poly_hl_model.py:60: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokens = torch.cat([torch.tensor(dataset.tokens) for dataset in datasets], dim=0)
/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/poly_hl_model.py:61: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.cat([torch.tensor(dataset.labels) for dataset in datasets], dim=0)
/Users/evananders/far_cluster/circuits-benchmark-saes/poly_bench/cases/utils.py:16: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sou

In [28]:
ll_model = poly_hl_model.get_ll_model()
corr = poly_hl_model.get_correspondence()
for k, v in corr.items():
    print(k, v)

n_epochs = 100
training_args = {
    "batch_size": 256,
    "lr": 1e-3,
    "num_workers": 0,
    "use_single_loss": False,
    "behavior_weight": 1., #basically doubles the strict weight's job.
    "iit_weight": 1.,
    "strict_weight": 0.4,
    "clip_grad_norm": 1.0,
    "iit_weight_schedule" : lambda s, i: s,
    "strict_weight_schedule" : lambda s, i: s,
    "behavior_weight_schedule" : lambda s, i: s, #0.955*s if 0.955**i > 0.01 else s, #have behavior weight decay over time
    "early_stop" : True,
    "lr_scheduler": torch.optim.lr_scheduler.LinearLR,
    "scheduler_kwargs": dict(start_factor=1, end_factor=0, total_iters=n_epochs),
    "scheduler_val_metric": ["val/accuracy", "val/IIA"], #for ReduceLRonPlateau
    "scheduler_mode": "max", #for ReduceLRonPlateau
}
train_set, test_set = poly_dataset.get_IIT_train_test_set()
model_pair = StrictIITModelPair(hl_model=poly_hl_model, ll_model=ll_model, corr=corr, training_args=training_args)
print(model_pair.nodes_not_in_circuit)
model_pair.train(
    train_set=train_set,
    test_set=test_set,
    optimizer_cls=torch.optim.AdamW,
    epochs=n_epochs,
    optimizer_kwargs=dict(weight_decay=1e-4),
)

input_hook {LLNode(name='blocks.0.hook_resid_pre', index=[:], subspace=None)}
mlp_hooks.0 {LLNode(name='blocks.0.mlp.hook_post', index=[:], subspace=None)}
attn_hooks.0.0 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 0, :], subspace=None)}
attn_hooks.0.1 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 1, :], subspace=None)}
task_hook {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 2, :], subspace=None)}
attn_hooks.0.3 {LLNode(name='blocks.0.attn.hook_z', index=[:, :, 3, :], subspace=None)}
mlp_hooks.1 {LLNode(name='blocks.1.mlp.hook_post', index=[:], subspace=None)}
mlp_hooks.2 {LLNode(name='blocks.2.mlp.hook_post', index=[:], subspace=None)}
attn_hooks.2.3 {LLNode(name='blocks.2.attn.hook_z', index=[:, :, 3, :], subspace=None)}
[LLNode(name='blocks.1.attn.hook_z', index=[:, :, 0, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 1, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', index=[:, :, 2, :], subspace=None), LLNode(name='blocks.1.attn.hook_z', 

Epoch 1: lr: 9.90e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 4.00e-01, train/iit_loss: 8.93e-01, train/behavior_loss: 6.24e-01, train/strict_loss: 1.07e-01, val/iit_loss: 5.74e-01, val/IIA: 77.00, val/accuracy: 90.65, val/strict_accuracy: 88.72
Epoch 2: lr: 9.80e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 4.00e-01, train/iit_loss: 6.71e-01, train/behavior_loss: 2.23e-01, train/strict_loss: 4.12e-02, val/iit_loss: 7.05e-01, val/IIA: 74.95, val/accuracy: 91.16, val/strict_accuracy: 89.50
Epoch 3: lr: 9.70e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 4.00e-01, train/iit_loss: 5.56e-01, train/behavior_loss: 1.80e-01, train/strict_loss: 3.09e-02, val/iit_loss: 4.37e-01, val/IIA: 82.31, val/accuracy: 93.75, val/strict_accuracy: 92.17
Epoch 4: lr: 9.60e-04, iit_weight: 1.00e+00, behavior_weight: 1.00e+00, strict_weight: 4.00e-01, train/iit_loss: 3.65e-01, train/behavior_loss: 1.58e-01, train/strict_loss: 3.26e-02, val/